## Visualizing the Allen Institute .swc morphologies with different customizations

In [ ]:
%matplotlib inline
from ateamopt.morph_handler import MorphHandler,swc_dict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ateamopt.utils import utility
from allensdk.core.cell_types_cache import CellTypesCache
import pandas as pd
import os
from ateamopt.animation import animation_module
import ateamopt
import warnings
import neurom.viewer

warnings.filterwarnings('ignore')

In [ ]:
def get_morph_path(cell_id,save_filename=None):
    try:
        from ateam.data import lims
        lr = lims.LimsReader()
        morph_path = lr.get_swc_path_from_lims(int(cell_id))
    except:
        morph_path = save_filename or '%s.swc'%cell_id
        ctc = CellTypesCache()
        ctc.get_reconstruction(int(cell_id),file_name=morph_path)
    return morph_path 

In [ ]:
cell_id = '479225052'
color_crimson=np.array([193,48,78,200])/255
color_steelblue = np.array([86,130,164,255])/255

swc_sect_indices = list(swc_dict.keys())
color_dict = {swc_sect_indx:color_steelblue for swc_sect_indx in swc_sect_indices}
morph_path = get_morph_path(cell_id)

In [ ]:
fig,ax = neurom.viewer.draw(neurom.load_neuron(morph_path))
fig.set_size_inches(4,10)
ax.grid(False)
ax.set_title('')
sns.despine(ax=ax)
ax.set_xlabel('$\mu m$')
ax.set_ylabel('$\mu m$')
fig.savefig('morph2D_neurom.png',bbox_inches='tight')

In [ ]:
morph_handler = MorphHandler(morph_path)
morph_data,morph_apical,morph_axon,morph_dist_arr = morph_handler.get_morph_coords()               
theta,axis_of_rot = morph_handler.calc_rotation_angle(morph_data,morph_apical)

sns.set(style='whitegrid')
title_fontsize=14
fig = plt.figure(figsize=(14,10))
ax1 = fig.add_subplot(1, 3, 1, projection='3d')
ax1,elev_angle = morph_handler.draw_morphology(theta,axis_of_rot,reject_axon=False,ax=ax1)
ax1.set_title('default colors',fontsize= title_fontsize)
ax1.patch.set_facecolor('white')
ax1.xaxis.set_major_formatter(plt.NullFormatter())
ax1.yaxis.set_major_formatter(plt.NullFormatter())
ax1.zaxis.set_major_formatter(plt.NullFormatter())

ax2 = fig.add_subplot(1, 3, 2, projection='3d')
ax2,elev_angle = morph_handler.draw_morphology(theta,axis_of_rot,reject_axon=False,ax=ax2,
                                               color_dict=color_dict,axis_off=True)
ax2.set_title('custom colors',fontsize= title_fontsize)
ax3 = fig.add_subplot(1, 3, 3, projection='3d')
ax3,elev_angle = morph_handler.draw_morphology(theta,axis_of_rot,reject_axon=False,ax=ax3,
                                               morph_dist_arr=morph_dist_arr,axis_off=True,alpha=.8)
ax3.set_title('thinning with distances',fontsize= title_fontsize)
fig.subplots_adjust(wspace=0)
plt.show()
fig.savefig('3D_morph_banner.png',bbox_inches='tight')

In [ ]:
ax,elev_angle = morph_handler.draw_morphology(theta,axis_of_rot,reject_axon=False,
                                              morph_dist_arr=morph_dist_arr,axis_off=True,alpha=.8)
n_syn_apical = 10
ax = morph_handler.add_synapses(morph_apical,n_syn_apical,theta,axis_of_rot,ax,color='k')
ax.set_title('morphology with synapses',fontsize=12)
fig = ax.get_figure()
fig.set_size_inches(4,10)
fig.savefig('Morph_with_synapses.png',bbox_inches='tight')
plt.show()


In [ ]:
angles = np.linspace(0,360,30)[:-1] # rotate the viewing position

files = animation_module.Animation.make_3Dviews(ax,angles,elevation=elev_angle,prefix='morph_anim/tmprot_')
movie_name = 'morph_movie_synapses.gif'
anim = animation_module.Animation(movie_name)
anim.make_gif(files,delay=20)


### Plotting in 2D

In [ ]:
morph_path = get_morph_path(cell_id)
morph_handler = MorphHandler(morph_path)
morph_data,morph_apical,morph_axon,morph_dist_arr = morph_handler.get_morph_coords()                            
theta,axis_of_rot = morph_handler.calc_rotation_angle(morph_data,morph_apical)
sns.set(style='whitegrid')
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(12,8))
ax1 = morph_handler.draw_morphology_2D(theta,axis_of_rot,reject_axon=False,ax=ax1)
ax1.set_title('default colors',fontsize=12)
ax1.set_xlabel('$\mu m$')
ax1.set_ylabel('$\mu m$')
sns.despine(ax=ax1)
ax2 = morph_handler.draw_morphology_2D(theta,axis_of_rot,reject_axon=False,ax=ax2,
                                               color_dict=color_dict,axis_off=True)
ax2.set_title('custom colors',fontsize=12)
ax3 = morph_handler.draw_morphology_2D(theta,axis_of_rot,reject_axon=False,ax=ax3,
                                               morph_dist_arr=morph_dist_arr,axis_off=True)
ax3.set_title('thinning with distances',fontsize=12)
fig.savefig('2D_morph_banner.png',bbox_inches='tight')
plt.show()

### Comparison between Mouse and Human

In [ ]:
human_cell_id,mouse_cell_id = '571735073','479225052'
cell_ids = [mouse_cell_id,human_cell_id]
soma_displacement_x = 500
color_dict = {4: 'purple', 3: 'r', 2: 'b', 1 :'k' }
fig,ax = plt.subplots(figsize=(6,10))

for jj,cell_id in enumerate(cell_ids):
    morph_path = get_morph_path(cell_id)       
    morph_handler = MorphHandler(morph_path)
    morph_data,morph_apical,morph_axon,morph_dist_arr = morph_handler.get_morph_coords()
    soma_loc = np.array([jj*soma_displacement_x, 0])
    
    # Rotate morphology to appear upright                            
    theta,axis_of_rot = morph_handler.calc_rotation_angle(morph_data,morph_apical)

    ax = morph_handler.draw_morphology_2D(theta,axis_of_rot,soma_loc=soma_loc,color_dict=color_dict,morph_dist_arr=morph_dist_arr,ax=ax,lw=1.2,reject_axon=True)  
ax.set_xlabel('$\mu m$')
ax.set_ylabel('$\mu m$')
ax.text(150,300,'Mouse',rotation='vertical',fontsize=14)
ax.text(650,600,'Human',rotation='vertical',fontsize=14)

ax.grid(False)
sns.despine(ax=ax)
fig.savefig('mouse_vs_human.png',bbox_inches='tight')

### Multiple morphologies in Laminar view 2D

In [ ]:
data_path = os.path.join(os.path.dirname(ateamopt.__file__),os.pardir,'assets','aggregated_data')
cre_coloring_filename = os.path.join(data_path,'rnaseq_sorted_cre.pkl')
cre_color_dict = utility.load_pickle(cre_coloring_filename)
cre_color_dict['Other'] = (0,0,0)

depth_data_filename = os.path.join(data_path,'mouse_me_and_met_avg_layer_depths.json') # Average layerwise depths for mouse
depth_data = utility.load_json(depth_data_filename)
total_depth = depth_data['wm']

# Cells are chosen to sample from diverse types within each layer
cell_id_dict = {'1':['574734127','564349611','475585413','555341581','536951541'],
                '2/3':['485184849','475515168','485468180','476087653','571306690'],
                '4':['483101699','602822298','490205998','569723367','324257146'],
                '5':['479225052','607124114','515249852','488683425','569998790'],
                '6a':['490259231','473564515','561985849','578938153','471789504'],
                '6b':['589128331','574993444','510136749','509881736','590558808']}

# Get normalized depth metadata for individual cells

ctc = CellTypesCache()
cells_allensdk = ctc.get_cells(species = ['Mus musculus'],simple = False)
sdk_data = pd.DataFrame(cells_allensdk)
sdk_data['specimen__id'] = sdk_data['specimen__id'].astype(str)


In [ ]:
ylim_min,ylim_max =-200, 1200
soma_loc_x = 0
sigma_layer = 50
soma_loc_displacement_x = 500
unique_layers = sorted(sdk_data.structure__layer.unique().tolist())

layer_dist = {layer_ : i*soma_loc_displacement_x for i,layer_ in enumerate(unique_layers)}

sns.set(style='whitegrid')
fig,ax = plt.subplots()

for layer_name,cell_id_list in cell_id_dict.items():
    num_cells = len(cell_id_list)
    dist_x = np.linspace(-125,125,num_cells)
    for kk,cell_id in enumerate(cell_id_list):
        metadata= sdk_data.loc[sdk_data.specimen__id==cell_id,['csl__normalized_depth','line_name',
                                                               'structure__layer']]
        norm_depth = metadata.csl__normalized_depth.tolist()[0]
        cre = metadata.line_name.tolist()[0]
        layer = metadata.structure__layer.tolist()[0]
        color = cre_color_dict[cre] if cre in cre_color_dict.keys() else cre_color_dict['Other']
        color_dict = {swc_sect_indx:color for swc_sect_indx in range(1,5)}
    
        loc_x = layer_dist[layer]+dist_x[kk]
        soma_loc = np.array([loc_x,(1-norm_depth)*total_depth])
        morph_path = get_morph_path(cell_id)
        
        morph_handler = MorphHandler(morph_path)
        morph_data,morph_apical,morph_axon,morph_dist_arr = morph_handler.get_morph_coords()
        
        # Rotate morphology to appear upright                            
        theta,axis_of_rot = morph_handler.calc_rotation_angle(morph_data,morph_apical)
        
        ax = morph_handler.draw_morphology_2D(theta,axis_of_rot,reject_axon=True,
                                  soma_loc=soma_loc,color_dict=color_dict,
                                  morph_dist_arr=morph_dist_arr,ax=ax)  
        
        
ax.set_ylim([ylim_min,ylim_max])
xmin,xmax = ax.get_xlim()
xmin += 200
xmax -= 200
for layer_ in unique_layers:
    layer_height = total_depth if layer_ == '1' else total_depth-depth_data[layer_] 
    ax.hlines(layer_height,xmin,xmax,colors='grey',lw=.5,linestyles='dashed')

# Add scale
ax.plot([-50,50],[total_depth-depth_data['5'], total_depth-depth_data['5']],
        lw=1,color='k')
ax.text(0, total_depth-depth_data['5']-80, '$100\:\mu m$',fontsize=10,
        horizontalalignment='center')
ax.plot([-50,-50],[total_depth-depth_data['5'], total_depth-depth_data['5']+200],
        lw=1,color='k')
ax.text(-110, total_depth-depth_data['5']+100, '$200\:\mu m$',fontsize=10,
        verticalalignment='center',rotation=90)

ax.hlines(0,xmin,xmax,colors='grey',lw=.25,linestyles='dashed')
ax.axis('off')
fig.set_size_inches(24,12) 
fig.savefig('morph_layerwise.png')

plt.show()